In [ ]:
from glob import glob
import platform
import os
if platform.system()=='Linux':
  from google.colab import drive
  drive.mount('/content/drive')
  os.chdir('/content/drive/My Drive/')
  sep='/'
else:
  sep='\\'
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive


In [ ]:
import sqlite3 as lite

In [ ]:
con = lite.connect('Team3/copy.sqlite')

In [ ]:
def transact(con, sql):
    try:
        con.execute(sql)
        con.commit()
    except Exception as e:
        con.rollback()
        print(e)

In [ ]:
import pandas as pd

**READING DATA FROM THE SPREADSHEET**

In [ ]:
#bb = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='bankbalance')
#Userz = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='Users')
#catgor = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='Category')
budgett = pd.read_excel("Team3/transaction_data.xlsx",sheet_name='Budget')


**CREATING TABLE Users**

In [ ]:
# con.execute("DROP TABLE IF EXISTS Users")
con.execute("""CREATE TABLE IF NOT EXISTS Users (
                username TEXT PRIMARY KEY,
                name TEXT NOT NULL,
                email TEXT UNIQUE CHECK (email LIKE '_%@_%._%'),
                pin TEXT
            )""")
con.commit()


In [ ]:
# con.execute("Drop table if exists Category")
con.execute("""CREATE TABLE if not exists Category (
    category_id text primary key,
    category_name TEXT NOT NULL
);
""")
con.commit()

In [ ]:
# con.execute("Drop table if exists Budget")
con.execute("""CREATE TABLE if not exists Budget (
    budget_id INTEGER PRIMARY KEY autoincrement,
    username text,
    category_id text,
    budget_amount REAL
    ,total_amount real,
    start_date text,
    end_date text,
    FOREIGN KEY (username) REFERENCES Users(username),
    FOREIGN KEY (category_id) REFERENCES Categories(category_id)
);

""")
con.commit()

In [ ]:
# con.execute("drop table if exists balance")
con.execute("""CREATE TABLE if not exists balance (
    bankbalance_id INTEGER PRIMARY KEY AUTOINCREMENT,
    username text,
    bankbalance int check (bankbalance>=0),
    FOREIGN KEY (username) REFERENCES Users(username)
);
""")
con.commit()

In [ ]:
import hashlib
def encrypt(pin):
  return hashlib.sha256(str(pin).encode()).hexdigest()

In [ ]:
# for row in Userz.itertuples(index=False):
#     encrypted_pin = encrypt(row.pin)
#     con.execute("INSERT INTO Users (username, name, email, pin) VALUES (?, ?, ?, ?)",
#                    (row.username, row.name, row.email, encrypted_pin))

In [ ]:
con.commit()

In [ ]:
con.execute("select * from Users").fetchone()

('aarav_k01',
 'Aarav Kumar',
 'aaravk.bsc23@rvu.edu.in',
 'bcb15f821479b4d5772bd0ca866c00ad5f926e3580720659cc80d39c9d09802a')

# what is going on here
"""for row in userz.itertuples(index=False): This line starts a loop that iterates over each row in the userz DataFrame.
 itertuples() is a Pandas function that returns an iterator yielding namedtuples for each row in the DataFrame.
  Setting index=False ensures that the index of the DataFrame is not included in the tuples.
con.execute("INSERT INTO Users VALUES (?, ?, ?)", row): Within each iteration of the loop, this line executes an SQL INSERT statement
. It inserts a row into the "Users" table using the values from the current row of the DataFrame (row).
 The question marks (?) are placeholders for the values to be inserted.
 This is a parameterized query where values from the DataFrame row are substituted into the placeholders."""


In [ ]:
# for row in catgor.itertuples(index=False):
#     con.execute("INSERT INTO category VALUES (?,?)", row)
# con.commit()

In [ ]:
con.execute("select * from category").fetchone()

('CE', 'Casual Entertainment')

In [ ]:
# for row in bb.itertuples(index=False):
#     con.execute("INSERT INTO balance(username,bankbalance) VALUES (?,?)", row)
# con.execute("select * from balance").fetchall()

In [ ]:
con.execute("select * from Budget").fetchone()
z= budgett.iloc[:,:4]
z
for row in z.itertuples(index=False):
    con.execute("INSERT INTO Budget(username,category_id,budget_amount,total_amount,start_date,end_date) VALUES (?,?,?,?,'1-04-2024','30-04-20204')", row)
con.execute("select * from Budget").fetchall()
con.commit()

In [ ]:
import pytz
from datetime import datetime

# Get current time in Delhi timezone
delhi_timezone = pytz.timezone('Asia/Kolkata')
delhi_time = datetime.now(delhi_timezone).strftime('%H:%M:%S')
# con.execute("DROP TABLE IF EXISTS transactions")
con.execute("""CREATE TABLE IF NOT EXISTS transactions (
    transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
    senders_id TEXT NOT NULL,
    receivers_id TEXT NOT NULL,
    amount INTEGER CHECK(amount > 0),
    date TEXT DEFAULT (DATE(CURRENT_TIMESTAMP)),
    time TEXT,
    category_id TEXT,
    FOREIGN KEY (senders_id) REFERENCES Users(username) ON DELETE RESTRICT,
    FOREIGN KEY (receivers_id) REFERENCES Users(username) ON DELETE RESTRICT,
    FOREIGN KEY (category_id) REFERENCES Category(category_id)
);
""".format(delhi_time))

con.commit()

#current timestamp provides time and date, having date(timestamp) returns only the current date, same for time
#fk connects a column in the given table to another tables pk, this means u can access cols of the other connected table
#association of tables
#on insert restrict allows us to restrict change/insertion/deletion in the child table that violates the foreign key constraint

# a trigger works during crud ops, here the triggers r raised if the username in transactions is not present in users username
# tried using on insert restrict and DEFERRABLE INITIALLY DEFERRED didnt work therefore chose a trigger

#raise(abort) raises an error saying invalid id, of type abort(which halts the current sql statement and rollsback to the)
#previous state,

#SELECT 1 is to check for the existence of a row that meets certain conditions but you don't need to retrieve any actual data from
#the table. It's a way to check whether a condition is true without incurring the overhead of retrieving any specific columns.

In [ ]:
con.execute("""CREATE TRIGGER if not exists update_balance
AFTER INSERT ON transactions
BEGIN
    UPDATE balance
    SET bankbalance = bankbalance - NEW.amount
    WHERE username = NEW.senders_id;

    UPDATE balance
    SET bankbalance = bankbalance + NEW.amount
    WHERE username = NEW.receivers_id;
END;
""")

In [ ]:
con.execute("""
CREATE TRIGGER if not exists insert_timestamp
AFTER INSERT ON transactions
BEGIN
    UPDATE transactions
    SET date = DATE('now', 'localtime'), time = TIME('now', '+05:30')
    WHERE rowid = NEW.rowid;
END;
""")
con.commit()

Use cases of our transaction system.
1. send and receive transactions, the amount is automatically deducted from the sender and increased in the receiver
2. to send and receive transactions, the amount is to be present in the balance of the sender and the pin entered must match the pin stored in the database(prototype)
3. the database also checks if the sender and receiver are both registered in the users table
4. you can get a users bankbalance and transaction history by entering the pin
5. ask for username and password while logging in


In [ ]:
receiver_categories = {
    'AmazedByDeals2023': 'SHP',
    'FlipkartFanatic101': 'SHP',
    'FoodieBangalore99': 'F&B',
    'UberUser_BLR123': 'TRVL',
    'MetroCard_1234567890': 'TRVL',
    'PolarBear_Environmental': 'F&B',
    'Zomatoholic20': 'F&B',
    'Jio_SIM_1234567890': 'PRE',
    'Netflix_Buff': 'CE',
    'TravelExplorer55': 'TRVL',
    'LPG_Consumer_12345': 'DOM',
    'MingosFan_BLR': 'F&B',
    'ApolloHealth_12345': 'MED',
    'Bigb_2342':'GRC',
    'Sapna_house10':'EDU',
    'Bscom_12': 'ELEC',
    'Bhel_189': 'ELEC'
}

In [ ]:
cursor = con.cursor()

# Define a function to check if budget exceeded
def check_budget(sender_id, category_id, amount):
    # Fetch budget amount for the sender and category
    cursor.execute("SELECT budget_amount FROM Budget WHERE username = ? AND category_id = ?", (sender_id, category_id))
    budget_amount = cursor.fetchone()
    if budget_amount and amount > budget_amount[0]:
        print(budget_amount)
        print(amount)
        return True  # Budget exceeded
    else:
        return False  # Budget not exceeded

# Define a function to handle the trigger logic
def check_budget_trigger(sender_id, receiver_id, amount):
    # Check if receiver_id is in the predefined list
    if receiver_id in receiver_categories:
        rec_category = receiver_categories[receiver_id]
        # Check if the sender has set a budget for this category and if the transaction amount exceeds it
        if check_budget(sender_id, rec_category, amount):
            # Handle budget exceeded scenario (you can log or take any other action here)
            # For simplicity, I'm just printing a message
            print("Budget Exceeded!")
            return False  # Budget exceeded, don't insert into transactions table
    return True  # Budget not exceeded, proceed with insertion

In [ ]:
def update_budget(sender_id,reciver_id,amount):
    category_id = receiver_categories[reciver_id]
    x =con.execute(f"select budget_amount from Budget where username  = '{sender_id}' and  category_id = '{category_id}'").fetchone()[0]
    x-=amount
    con.execute(f"update Budget set budget_amount = {x} where username = '{sender_id}' and category_id = '{category_id}'")
    y  =con.execute(f"select total_amount from Budget where username ='{sender_id}' and category_id = '{category_id}'").fetchone()[0]
    y+=amount
    con.execute(f"update Budget set total_amount = {y} where username ='{sender_id}' and category_id = '{category_id}'")

    con.commit()
    print("Transaction inserted successfully!")

In [ ]:
def authenticate_user(username, pin):
    user = con.execute("SELECT username FROM Users WHERE username=? AND (pin=? OR pin IS NULL)", (username,pin)).fetchone()
    if user:
        return user[0]
    else:
        return None


In [ ]:
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

# Email configuration
EMAIL_ADDRESS = 'atheendreramesh@gmail.com'
EMAIL_PASSWORD = 'ddjq gtlz kgtk ggtk'
SMTP_SERVER = 'smtp.gmail.com'
SMTP_PORT = 587

def send_email(sender_email, receiver_email, subject, body):
    msg = MIMEMultipart('alternative')
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'html'))

    with smtplib.SMTP(SMTP_SERVER, SMTP_PORT) as s:
        s.starttls()
        s.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        s.send_message(msg)

In [ ]:
def transact2(senders_id, receivers_id, amount):
    if receivers_id is None:
        receivers_id = input("Please enter the receiver's username: ")
        amount = int(input("Please enter the amount: "))

    if senders_id != receivers_id:
        receiver_exists = con.execute("SELECT 1 FROM Users WHERE username=?", (receivers_id,)).fetchone()

        # makes sure you're not sending it to the same user
        if not receiver_exists:
            print("Receiver ID not found")
            return
        else:
            # Check if the sender has enough balance
            sender_balance = con.execute("SELECT bankbalance FROM balance WHERE username=?", (senders_id,)).fetchone()[0]
            if sender_balance < amount:
                print(f"Insufficient balance. Your current balance is {sender_balance}.")
                return

            category_id = None
            if receivers_id in receiver_categories:
              category_id = receiver_categories[receivers_id]

              if check_budget_trigger(senders_id, receivers_id, amount):
                  update_budget(senders_id, receivers_id, amount)
                  category_id = receiver_categories[receivers_id]

                  try:
                   con.execute("BEGIN")
                   con.execute("INSERT INTO transactions (senders_id, receivers_id, amount, category_id) VALUES (?, ?, ?, ?)",
                            (senders_id, receivers_id, amount, category_id))
                   con.commit()
                   print("Transaction successful.")

                # Send emails after successful transaction
                   sender_email = con.execute("SELECT email FROM Users WHERE username=?", (senders_id,)).fetchone()[0]
                   receiver_email = con.execute("SELECT email FROM Users WHERE username=?", (receivers_id,)).fetchone()[0]

                   sender_subject = "Transaction Successful"
                   sender_body = f"Your transaction of {amount} to {receivers_id} is successful."
                   send_email(EMAIL_ADDRESS, sender_email, sender_subject, sender_body)

                   receiver_subject = "You received a transaction"
                   receiver_body = f"You received {amount} from {senders_id}."
                   send_email(EMAIL_ADDRESS, receiver_email, receiver_subject, receiver_body)

                  except Exception as e:
                    con.rollback()
                    print(f"Error: {e}")
                    # Send email for transaction failure
                    sender_email = con.execute("SELECT email FROM Users WHERE username=?", (senders_id,)).fetchone()[0]
                    sender_subject = "Transaction Failed"
                    sender_body = "Your transaction failed. Please try again later."
                    send_email(EMAIL_ADDRESS, sender_email, sender_subject, sender_body)

            else:
               try:
                   con.execute("BEGIN")
                   con.execute("INSERT INTO transactions (senders_id, receivers_id, amount, category_id) VALUES (?, ?, ?, ?)",
                            (senders_id, receivers_id, amount, category_id))
                   con.commit()
                   print("Transaction successful.")

                # Send emails after successful transaction
                   sender_email = con.execute("SELECT email FROM Users WHERE username=?", (senders_id,)).fetchone()[0]
                   receiver_email = con.execute("SELECT email FROM Users WHERE username=?", (receivers_id,)).fetchone()[0]

                   sender_subject = "Transaction Successful"
                   sender_body = f"Your transaction of {amount} to {receivers_id} is successful."
                   send_email(EMAIL_ADDRESS, sender_email, sender_subject, sender_body)

                   receiver_subject = "You received a transaction"
                   receiver_body = f"You received {amount} from {senders_id}."
                   send_email(EMAIL_ADDRESS, receiver_email, receiver_subject, receiver_body)

               except Exception as e:
                    con.rollback()
                    print(f"Error: {e}")
                    # Send email for transaction failure
                    sender_email = con.execute("SELECT email FROM Users WHERE username=?", (senders_id,)).fetchone()[0]
                    sender_subject = "Transaction Failed"
                    sender_body = "Your transaction failed. Please try again later."
                    send_email(EMAIL_ADDRESS, sender_email, sender_subject, sender_body)


    else:
        print("Cannot transact to the same sender and receiver.")

# Checks for LOGIN


In [ ]:
import getpass

In [ ]:
# Failure if WRONG USERNAME/password
def login():
  username=input("pls enter username: ")
  pin=getpass.getpass("pls input pin: ")
  pin=encrypt(int(pin))
  x=authenticate_user(username,pin)
  if x:
    print("log in successful.")
    return username
  else:
    print("invalid username or pin.")

senders_id=login()

pls enter username: ram
pls input pin: ··········
invalid username or pin.


In [ ]:
# SUCCESS CASE WITH RIGHT USERNAME AND PASSWORD
def login():
  username=input("pls enter username: ")
  pin=getpass.getpass("pls input pin: ")
  pin=encrypt(int(pin))
  x=authenticate_user(username,pin)
  if x:
    print("log in successful.")
    return username
  else:
    print("invalid username or pin.")

senders_id=login()

pls enter username: Ramyashree_04
pls input pin: ··········
log in successful.


# Checks for Transaction


In [ ]:
#Failure case by Incorrect PIN
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Authentication failed. Incorrect username or PIN.


In [ ]:
# Failure case with wrong receiver's username
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: ram
Please enter the amount: 10
Receiver ID not found


In [ ]:
#Failure if Transaction exceeds Bank balance
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Atheendre_ram
Please enter the amount: 10000
Insufficient balance. Your current balance is 6610.


In [ ]:
# Success Case for Transaction from one user to another
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Atheendre_ram
Please enter the amount: 10
Transaction successful.


# Checks on Transactions of specific user on Budget set for Different categories

**TESTS ON TRAVEL(TRVL) CATEGORY**

In [ ]:
# Failure case (Set amt for TVRL category is 500)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: TravelExplorer55
Please enter the amount: 510
(500.0,)
510
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for TVRL category is 500)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: TravelExplorer55
Please enter the amount: 500
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for TVRL category is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: TravelExplorer55
Please enter the amount: 10
(0.0,)
10
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON GROCERIES(GRC) CATEGORY**

In [ ]:
# Failure case (Set amt for GRC category is 2000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Bigb_2342
Please enter the amount: 2001
(2000.0,)
2001
Budget Exceeded!


In [ ]:
con.commit()

In [ ]:
# Borderline success case (Set amt for GRC category is 2000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Bigb_2342
Please enter the amount: 2000
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for GRC category 2000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Bigb_2342
Please enter the amount: 10
(0.0,)
10
Budget Exceeded!


**TESTS ON MEDICINE(MED) CATEGORY**

In [ ]:
con.commit()

In [ ]:
# Failure case (Set amt for MED category is 1000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: ApolloHealth_12345
Please enter the amount: 1500
(80.0,)
1500
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for MED category is 1000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: ApolloHealth_12345
Please enter the amount: 80
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for MED category 1000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: ApolloHealth_12345
Please enter the amount: 12
(0.0,)
12
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON EDUCATION(EDU) CATEGORY**

In [ ]:
# Failure case (Set amt for EDU category is 1000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Sapna_house10
Please enter the amount: 1002
(1000.0,)
1002
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for EDU category is 1000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Sapna_house10
Please enter the amount: 1000
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for EDU category 1000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Sapna_house10
Please enter the amount: 10
(0.0,)
10
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON DOMESTIC(DOM) CATEGORY**

In [ ]:
# Failure case (Set amt for DOM category is 5000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: LPG_Consumer_12345
Please enter the amount: 50001
(5000.0,)
50001
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for DOM category is 5000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: LPG_Consumer_12345
Please enter the amount: 5000
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for DOM category 3000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: LPG_Consumer_12345
Please enter the amount: 44
(0.0,)
44
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON PREPAID(PRE) CATEGORY**

In [ ]:
# Failure case (Set amt for PRE category is 2500)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Jio_SIM_1234567890
Please enter the amount: 2501
(2500.0,)
2501
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for PRE category is 2500)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Jio_SIM_1234567890
Please enter the amount: 2500
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for PRE category 2500 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Jio_SIM_1234567890
Please enter the amount: 34
(0.0,)
34
Budget Exceeded!


**TESTS ON ELECTRICITY(ELEC) CATEGORY**

In [ ]:
con.commit()

In [ ]:
# Failure case (Set amt for ELEC category is 3000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Bscom_12
Please enter the amount: 3001
(3000.0,)
3001
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for ELEC category is 3000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Bscom_12
Please enter the amount: 3000
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for ELEC category 3000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Bscom_12
Please enter the amount: 234
(0.0,)
234
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON CASUAL ENTERTAINMENT(CE) CATEGORY**

In [ ]:
# Failure case (Set amt for CE category is 3000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Netflix_Buff
Please enter the amount: 3001
(3000.0,)
3001
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for CE category is 3000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Netflix_Buff
Please enter the amount: 3000
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for CE category 3000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: Netflix_Buff
Please enter the amount: 42
(0.0,)
42
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON FOOD(F&B) CATEGORY**

In [ ]:
# Failure case (Set amt for F&B category is 5000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: MingosFan_BLR
Please enter the amount: 5001
(5000.0,)
5001
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for F&B category is 5000)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: MingosFan_BLR
Please enter the amount: 5000
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for F&B category 5000 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: MingosFan_BLR
Please enter the amount: 457
(0.0,)
457
Budget Exceeded!


In [ ]:
con.commit()

**TESTS ON SHOPPING(SHP) CATEGORY**

In [ ]:
# Failure case (Set amt for SHP category is 500)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: FlipkartFanatic101
Please enter the amount: 3455
(500.0,)
3455
Budget Exceeded!


In [ ]:
# Borderline success case (Set amt for SHP category is 500)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: FlipkartFanatic101
Please enter the amount: 500
Transaction inserted successfully!
Transaction successful.


In [ ]:
# Failure case (Set amt for SHP category 500 is already spent)
def transaction(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        transact2(senders_id,None,None)
    else:
        print("Authentication failed. Incorrect username or PIN.")
transaction(senders_id)

Please enter pin: ··········
Please enter the receiver's username: FlipkartFanatic101
Please enter the amount: 678
(0.0,)
678
Budget Exceeded!


In [ ]:
con.commit()

# To check balance using PIN


In [ ]:
#Failure case if Entered Wrong PIN
def getbankbalance(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    # Authenticate user
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        result = con.execute("SELECT bankbalance FROM balance WHERE username = ?", (senders_id,)).fetchone()
        if result:
            return result[0]
        else:
            return "User balance not found"
    else:
        return "Authentication failed. Incorrect username or PIN."
print("Your bank balance is:", getbankbalance(senders_id))


Please enter pin: ··········
Your bank balance is: Authentication failed. Incorrect username or PIN.


In [ ]:
#Success case if Entered PIN correctly
def getbankbalance(senders_id):
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    # Authenticate user
    user_id = authenticate_user(senders_id,pin)
    if user_id:
        result = con.execute("SELECT bankbalance FROM balance WHERE username = ?", (senders_id,)).fetchone()
        if result:
            return result[0]
        else:
            return "User balance not found"
    else:
        return "Authentication failed. Incorrect username or PIN."
print("Your bank balance is:", getbankbalance(senders_id))


Please enter pin: ··········
Your bank balance is: 76988


# History of the Transactions of a user



In [ ]:
def transaction_history():
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))

    user_id = authenticate_user(senders_id,pin)
    if user_id:
        result = con.execute("SELECT senders_id, receivers_id, amount, date, time,category_id FROM transactions WHERE senders_id = ? OR receivers_id = ?", (senders_id,senders_id )).fetchall()
        if result:
            for transaction in result:
                print(f"Sender: {transaction[0]}, Receiver: {transaction[1]}, Amount: {transaction[2]}, Date: {transaction[3]}, Time: {transaction[4]}, category: {transaction[5]}")
        else:
            print(f"No transactions found for user {senders_id}")
    else:
        print("Authentication failed. Incorrect username or PIN.")

transaction_history()


Please enter pin: ··········
Sender: Ramyashree_04, Receiver: ApolloHealth_12345, Amount: 10, Date: 2024-05-04, Time: 18:47:10, category: MED
Sender: Ramyashree_04, Receiver: ApolloHealth_12345, Amount: 10, Date: 2024-05-04, Time: 18:47:42, category: MED
Sender: Atheendre_ram, Receiver: Ramyashree_04, Amount: 1, Date: 2024-05-04, Time: 18:54:42, category: None
Sender: Atheendre_ram, Receiver: Ramyashree_04, Amount: 10, Date: 2024-05-04, Time: 21:04:33, category: None
Sender: Ramyashree_04, Receiver: Atheendre_ram, Amount: 10, Date: 2024-05-04, Time: 23:46:10, category: None
Sender: Ramyashree_04, Receiver: TravelExplorer55, Amount: 500, Date: 2024-05-04, Time: 23:54:36, category: TRVL
Sender: Ramyashree_04, Receiver: Bigb_2342, Amount: 2100, Date: 2024-05-04, Time: 23:58:40, category: None
Sender: Ramyashree_04, Receiver: Bigb_2342, Amount: 2001, Date: 2024-05-04, Time: 01:41:14, category: None
Sender: Aasish_02, Receiver: Ramyashree_04, Amount: 50000, Date: 2024-05-04, Time: 01:49:54,

# Autopay

In [ ]:
#failure case of autopay
def autopay(senders_id):
    import time
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id=authenticate_user(senders_id,pin)
    receivers_id=input("pls enter receivers_id: ")
    amount=int(input("pls enter amount: "))
    if user_id:
        transact2(senders_id,receivers_id,amount)
        while True:
            print("autopay every 30 days.")
            time.sleep(30)#30 seconds just to show
            transact2(senders_id, receivers_id, amount)
            choice = input("Do you want to end the transaction? (yes/no): ")
            if choice.lower() == "yes":
                break
    else:
        print("Authentication failed. Incorrect username or PIN.")
autopay(senders_id)

Please enter pin: ··········
pls enter receivers_id: Netflix_Buff
pls enter amount: 200
(0.0,)
200
Budget Exceeded!
autopay every 30 days.
(0.0,)
200
Budget Exceeded!
Do you want to end the transaction? (yes/no): yes


In [ ]:
#success case of autopay
def autopay(senders_id):
    import time
    pin = getpass.getpass("Please enter pin: ")
    pin = encrypt(int(pin))
    user_id=authenticate_user(senders_id,pin)
    receivers_id=input("pls enter receivers_id: ")
    amount=int(input("pls enter amount: "))
    if user_id:
        transact2(senders_id,receivers_id,amount)
        while True:
            print("autopay every 30 days.")
            time.sleep(30)#30 seconds just to show
            transact2(senders_id, receivers_id, amount)
            choice = input("Do you want to end the transaction? (yes/no): ")
            if choice.lower() == "yes":
                break
    else:
        print("Authentication failed. Incorrect username or PIN.")
autopay(senders_id)

Please enter pin: ··········
pls enter receivers_id: Atheendre_ram
pls enter amount: 10
Transaction successful.
autopay every 30 days.
Transaction successful.
Do you want to end the transaction? (yes/no): no
autopay every 30 days.
Transaction successful.
Do you want to end the transaction? (yes/no): yes


In [ ]:
def alltransactions():
  return con.execute("select * from transactions").fetchall()
alltransactions()

[(1,
  'Atheendre_ram',
  'ApolloHealth_12345',
  10,
  '2024-05-04',
  '18:45:13',
  'MED'),
 (2,
  'Atheendre_ram',
  'ApolloHealth_12345',
  10,
  '2024-05-04',
  '18:45:47',
  'MED'),
 (3,
  'Atheendre_ram',
  'ApolloHealth_12345',
  10,
  '2024-05-04',
  '18:46:28',
  'MED'),
 (4,
  'Ramyashree_04',
  'ApolloHealth_12345',
  10,
  '2024-05-04',
  '18:47:10',
  'MED'),
 (5,
  'Ramyashree_04',
  'ApolloHealth_12345',
  10,
  '2024-05-04',
  '18:47:42',
  'MED'),
 (6, 'Atheendre_ram', 'Ramyashree_04', 1, '2024-05-04', '18:54:42', None),
 (7, 'Skanda_Pv', 'Atheendre_ram', 10, '2024-05-04', '19:01:36', None),
 (8, 'Atheendre_ram', 'Ramyashree_04', 10, '2024-05-04', '21:04:33', None),
 (9, 'Ramyashree_04', 'Atheendre_ram', 10, '2024-05-04', '23:46:10', None),
 (10,
  'Ramyashree_04',
  'TravelExplorer55',
  500,
  '2024-05-04',
  '23:54:36',
  'TRVL'),
 (11, 'Ramyashree_04', 'Bigb_2342', 2100, '2024-05-04', '23:58:40', None),
 (12, 'Atheendre_ram', 'Bigb_2342', 10, '2024-05-04', '00:14:

In [ ]:
pd.read_sql("SELECT * FROM sqlite_master where type='table'",con)

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,9,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Users,Users,2,CREATE TABLE Users (\n username...
2,table,Category,Category,5,CREATE TABLE Category (\n category_id text ...
3,table,Budget,Budget,4,CREATE TABLE Budget (\n budget_id INTEGER P...
4,table,balance,balance,7,CREATE TABLE balance (\n bankbalance_id INT...
5,table,transactions,transactions,11,CREATE TABLE transactions (\n transaction_i...
6,table,people,people,17,CREATE TABLE people (\n id INTEGER ...


In [ ]:
con.commit()

In [ ]:
con.close()